In [27]:
import pandas as pd
import os
import re
import sys
import numpy as np
from merf import MERF
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 
sns.set_context("poster")
from sklearn.ensemble import RandomForestRegressor
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.merf import MERF
from sklearn.model_selection import train_test_split, KFold
from merf.viz import plot_merf_training_stats

current_dir = os.getcwd() # Get the current working directory
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from em_utils import * # import the utils

# Create output directory if it doesn't exist
output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_plots/5.combined"
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/5.combined"
os.makedirs(output_dir, exist_ok=True)

### Read in data

In [ ]:
test = read_data(df_dir, "feb20_test_merged_all_omics_raw_meta.csv")
train = read_data(df_dir, "feb20_training_merged_all_omics_raw_meta.csv")

print("train columns: ", train.columns.to_list())

In [ ]:
print("---------- Select predictors for training set ----------")
train_set = train
train_set = train_set.drop(['record_id', 'subject_id','sample_id'], axis=1)
X = train_set.drop(['all_samples','time','outcome_BMI_fnl'], axis=1)

#X = X.drop(columns=['Unnamed: 0_tax', 'x_t'], errors='ignore')
Y = train_set[['outcome_BMI_fnl']]
Y = Y['outcome_BMI_fnl'].to_numpy() # Convert Y to numeric array
clusters_train = train_set['all_samples'].to_numpy() # Get ID variables
Z = np.ones((train_set.shape[0], 1)) # Create random effects matrix with ones

print("X shape: ", X.shape)
print("X data: \n", X)

In [ ]:
print("id variables : ", clusters_train)
print("Number of unique IDs: ", len(np.unique(clusters_train)))  # Print the number of unique IDs

In [ ]:
print("Y shape: ", Y.shape)
print("Y data: \n", Y)

In [ ]:
print("Z shape: ", Z.shape)
print("Z data: \n", Z)

In [ ]:
print("train columns: ", test.columns.to_list())

In [ ]:
print("---------- Select predictors for test set ----------")
test_set = test
test_set = test_set.drop(['record_id', 'subject_id','sample_id'], axis=1)
X_test = train_set.drop(['all_samples','time','outcome_BMI_fnl'], axis=1)
X_test = X_test[X.columns]  # Reorder and select columns to match training set
X_test = X_test.astype(X.dtypes)  # Ensure data types match

Y_test = test_set['outcome_BMI_fnl'].to_numpy()  # Convert Y to numeric array
clusters_test = pd.Series(test_set['all_samples'])  # Convert to pandas Series
# Create random effects matrix with ones
Z_test = np.ones((len(X_test), 1))

print("X_new shape: ", Y_test.shape)
print("X_new data: \n", Y_test)

# Run the training loop 

In [ ]:
# Hyperparameters to tune
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [None],
    'min_samples_split': [0.05, 0.1, 0.15],
    'max_iter': [2, 10],
    'n_splits': [3, 5, 10]  # Added n_splits for cross-validation
}

train_set = train_set
columns_to_drop = ['all_samples','time','outcome_BMI_fnl','Unnamed: 0_merged_data']

# Create training features
X = train_set.drop(columns=columns_to_drop, errors='ignore')
y = train_set[['outcome_BMI_fnl']]
y = y['outcome_BMI_fnl'].to_numpy() # Convert Y to numeric array
clusters = train_set['all_samples'].to_numpy() # Get ID variables
z = np.ones((train_set.shape[0], 1)) # Create random effects matrix with ones

print("X shape:", X.shape)
print("y shape:", y.shape)
print("clusters shape:", clusters.shape)
print("z shape:", z.shape)

best_score = float('inf')
best_params = {}
results = [] # Initialize a list to store the results of each iteration

# Loop through all possible combinations of parameters
for params in itertools.product(*param_grid.values()):
        n_estimators, max_depth, min_samples_split, max_iter, n_splits = params
        print(f"Combination: {params}\n")
        scores = []
        prev = []
        ptev = []
        oob_scores = []  # Initialize a list to store OOB scores

        # K-fold cross-validation with variable n_splits
        kf = KFold(n_splits=n_splits)
        for train_index, test_index in kf.split(X):
            print("Train indices:", train_index)
            print("Test indices:", test_index)
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Use .iloc for row selection
            y_train, y_test = y[train_index], y[test_index]
            clusters_train, clusters_test = clusters[train_index], pd.Series(clusters[test_index])
            z_train, z_test = z[train_index], z[test_index]
            model = MERF(
                # Specify the fixed effects model as a Random Forest Regressor
                fixed_effects_model=RandomForestRegressor(
                    n_estimators=n_estimators,  # Number of trees in the forest
                    max_depth=max_depth,  # Maximum depth of each tree
                    min_samples_split=min_samples_split,  # Minimum samples required to split an internal node
                    n_jobs=1,  # Number of jobs to run in parallel
                    oob_score=True  # Whether to use out-of-bag samples to estimate the R^2 on unseen data
                ),
                # Generalized Linear Model (GLM) early stopping threshold
                gll_early_stop_threshold=None,  # No early stopping threshold set
                # Maximum number of iterations for the MERF algorithm
                max_iterations=max_iter  # Maximum number of iterations to run the MERF algorithm
            )
            model.fit(X_train.select_dtypes(include=[np.number]), 
                      z_train, pd.Series(clusters_train), y_train)
            
            y_pred = model.predict(X_test, z_test, clusters_test)
            scores.append(np.mean((y_pred - y_test) ** 2)) # MSE
            
            # Calculate ptev and prev
            total_variance = np.var(y_test) #calculates the total variance of the predicted values
            random_effect_variance = np.var(y_test - y_pred)  # Variance of residuals
            fixed_effect_variance = total_variance - random_effect_variance

            ptev.append(np.mean(fixed_effect_variance / total_variance if total_variance > 0 else 0))
            prev.append(np.mean(random_effect_variance / total_variance if total_variance > 0 else 0))

            # Calculate OOB score
            forest = model.trained_fe_model
            oob_score = round(forest.oob_score_*100, 1)  # percent variation
            oob_scores.append(oob_score)  # Append OOB score to the list

            # Print ptev, prev, and OOB score for the current iteration
            print(f"Combination, ptev: {np.mean(ptev):.4f}, prev: {np.mean(prev):.4f}, OOB Score: {oob_score:.4f}")

        # Calculate the mean of the scores for the current combination of parameters
        mean_score = np.mean(scores)
        mean_prev = np.mean(prev)
        mean_ptev = np.mean(ptev)
        mean_oob_score = np.mean(oob_scores)  # Calculate the mean of OOB scores
        if mean_score < best_score:
            best_score = mean_score
            best_params = params

        # Append the results of the current iteration to the results list
        # Create a result dictionary with individual scores and mean scores
        result_dict = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'max_iter': max_iter,
            'n_splits': n_splits,  # Added n_splits to the result dictionary
            'mse_score_1': scores[0] if len(scores) > 0 else None,
            'mse_score_2': scores[1] if len(scores) > 1 else None,
            'mse_score_3': scores[2] if len(scores) > 2 else None,
            'mse_score_4': scores[3] if len(scores) > 3 else None,
            'mse_score_5': scores[4] if len(scores) > 4 else None,
            'mse_score_6': scores[5] if len(scores) > 5 else None,
            'mse_score_7': scores[6] if len(scores) > 6 else None,
            'mse_score_8': scores[7] if len(scores) > 7 else None,
            'mse_score_9': scores[8] if len(scores) > 8 else None,
            'mse_score_10': scores[9] if len(scores) > 9 else None,
            'mean_mse_score': mean_score,
            'prev_1': prev[0] if len(prev) > 0 else None,
            'prev_2': prev[1] if len(prev) > 1 else None,
            'prev_3': prev[2] if len(prev) > 2 else None,
            'prev_4': prev[3] if len(prev) > 3 else None,
            'prev_5': prev[4] if len(prev) > 4 else None,
            'prev_6': prev[5] if len(prev) > 5 else None,
            'prev_7': prev[6] if len(prev) > 6 else None,
            'prev_8': prev[7] if len(prev) > 7 else None,
            'prev_9': prev[8] if len(prev) > 8 else None,
            'prev_10': prev[9] if len(prev) > 9 else None,
            'mean_prev': mean_prev,
            'ptev_1': ptev[0] if len(ptev) > 0 else None,
            'ptev_2': ptev[1] if len(ptev) > 1 else None,
            'ptev_3': ptev[2] if len(ptev) > 2 else None,
            'ptev_4': ptev[3] if len(ptev) > 3 else None,
            'ptev_5': ptev[4] if len(ptev) > 4 else None,
            'ptev_6': ptev[5] if len(ptev) > 5 else None,
            'ptev_7': ptev[6] if len(ptev) > 6 else None,
            'ptev_8': ptev[7] if len(ptev) > 7 else None,
            'ptev_9': ptev[8] if len(ptev) > 8 else None,
            'ptev_10': ptev[9] if len(ptev) > 9 else None,
            'mean_ptev': mean_ptev,
            'oob_1': oob_scores[0] if len(oob_scores) > 0 else None,
            'oob_2': oob_scores[1] if len(oob_scores) > 1 else None,
            'oob_3': oob_scores[2] if len(oob_scores) > 2 else None,
            'oob_4': oob_scores[3] if len(oob_scores) > 3 else None,
            'oob_5': oob_scores[4] if len(oob_scores) > 4 else None,
            'oob_6': oob_scores[5] if len(oob_scores) > 5 else None,
            'oob_7': oob_scores[6] if len(oob_scores) > 6 else None,
            'oob_8': oob_scores[7] if len(oob_scores) > 7 else None,
            'oob_9': oob_scores[8] if len(oob_scores) > 8 else None,
            'oob_10': oob_scores[9] if len(oob_scores) > 9 else None,
            'oob_score': mean_oob_score
        }
        # Append the result dictionary to the results list
        results.append(result_dict)

print("Best parameters:", best_params)
print("Best score:", best_score)



In [37]:
# Convert the results list to a DataFrame and save it to a CSV file
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/5.combined"
results_df = pd.DataFrame(results)
results_df.to_csv(f'{df_dir}/feb20_params_fine_tuning_results_all_omics.csv', index=False)